<a href="https://colab.research.google.com/github/skywalker0803r/Ricky/blob/master/%E7%8E%89%E5%B1%B1%E9%8A%80%E8%A1%8C%E6%AF%94%E8%B3%BD_catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/drive')
import warnings 
warnings.simplefilter('ignore')
import seaborn as sns
import matplotlib.pyplot as plt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
中文map = {'bacno':'歸戶帳號','txkey':'交易序號','locdt':'授權日期','loctm':'授權時間','cano':'交易卡號',
         'contp':'交易類別','etymd':'交易型態','mchno':'特店代號','acqic':'收單行代碼','mcc':'MCC_CODE',
         'conam':'交易金額-台幣(經過轉換)','ecfg':'網路交易註記','insfg':'分期交易註記','iterm':'分期期數',
         'stocn':'消費地國別','scity':'消費城市','stscd':'狀態碼','ovrlt':'超額註記碼','flbmk':'Fallback註記',
         'hcefg':'支付型態','csmcu':'消費地幣別','flg_3dsmk':'3DS交易註記','fraud_ind':'盜刷註記'}

# train

In [11]:
import pandas as pd
train = pd.read_csv("/content/drive/My Drive/玉山人工智慧比賽數據/train_預處理完.csv",index_col=0)
print(train.shape)
train.head()

(1521787, 23)


,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey,fraud_ind
0,6862,112785,37846,16158,5,0,0,0,0,0,5,0,0,33,61954,457,59034,0,0,102,0,516056,0
1,0,133951,45476,13693,5,0,0,2,0,0,0,0,0,9,38216,451,0,0,5795,102,0,4376,0
2,6862,15350,187354,16158,5,0,0,0,0,0,5,0,0,6,54640,457,59034,0,0,102,0,483434,0
3,6697,156492,29812,40413,5,62,0,5,0,0,5,0,0,5,62128,247,50185,0,3267,102,0,1407164,0
4,5959,105534,80881,25962,5,62,0,4,0,0,5,0,0,6,65231,263,93290,0,5795,102,0,1051004,0


# test

In [12]:
test = pd.read_csv("/content/drive/My Drive/玉山人工智慧比賽數據/test_預處理完.csv",index_col=0)
test_txkey = test["txkey"]
print(test.shape)
test.head()

(421665, 22)


,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey
0,6862,162489,116168,16158,5,0,0,0,0,0,5,0,0,102,77950,457,59060,0,0,102,0,592489
1,6862,162489,116168,16158,5,0,0,0,0,0,5,0,0,102,79549,457,59060,0,0,102,0,592452
2,6862,162489,116168,16158,5,0,0,0,0,0,5,0,0,100,60355,457,59060,0,0,102,0,590212
3,6862,162489,116168,16158,5,0,0,0,0,0,5,0,0,100,60296,457,59060,0,0,102,0,590209
4,6862,162489,116168,16158,5,0,0,0,0,0,5,0,0,102,77933,457,59060,0,0,102,0,592488


# 定義 features & cat_features & target

In [13]:
# 全部特徵
features = train.drop(['fraud_ind', # just target
                       'txkey', # just like index
                       'locdt',# just date
                       ],axis=1).columns.tolist()

# 原始特徵才是cat_features
cat_features = sorted(list(set(features)&set(中文map.keys())))

y_name = 'fraud_ind'

print(len(features),features)
print(len(cat_features),cat_features)
print(len([y_name]),[y_name])

20 ['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd', 'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'loctm', 'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd']
20 ['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd', 'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'loctm', 'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd']
1 ['fraud_ind']


# both / test

In [14]:
f = {}
for col in cat_features:
  both_value = set(train[col].unique())&set(test[col].unique())
  f[col] = len(both_value)/len(test[col].unique())
res = pd.DataFrame(f,index=['both/test']).T.sort_values(by='both/test',ascending=False)
res['中文'] = res.index.map(中文map)
res

,both/test,中文
stscd,1.000000,狀態碼
ovrlt,1.000000,超額註記碼
contp,1.000000,交易類別
ecfg,1.000000,網路交易註記
etymd,1.000000,交易型態
flbmk,1.000000,Fallback註記
flg_3dsmk,1.000000,3DS交易註記
insfg,1.000000,分期交易註記
iterm,1.000000,分期期數
loctm,0.990267,授權時間


# drop rare features

In [15]:
print(len(features))
try:
  features.remove('bacno')
  features.remove('cano')
except:
  pass
print(len(features))

20
18


In [16]:
print(len(cat_features))
try:
  cat_features.remove('bacno')
  cat_features.remove('cano')
except:
  pass
print(len(cat_features))

20
18


# 切分 train 跟 val

In [0]:
from sklearn.model_selection import KFold

kf = KFold(n_splits =5 ,random_state=42)
Folds = {}

for i,train_val_index in enumerate(kf.split(train.index)):
  Folds[i] = train_val_index

In [0]:
num_Fold = 0
train_index ,val_index = Folds[num_Fold]

In [0]:
val = train.loc[val_index,features+[y_name]]
train = train.loc[train_index,features+[y_name]]

# 切分 val_1,val_2

In [20]:
from sklearn.model_selection import train_test_split
val_1 ,val_2 = train_test_split(val[features+[y_name]] ,test_size=0.5 ,random_state=42)
print(train.shape)
print(val_1.shape)
print(val_2.shape)

(1217429, 19)
(152179, 19)
(152179, 19)


# 計算樣本權重

In [21]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

y_org = train[y_name].append(val[y_name])

class_weight_dict = dict(zip([0,1],
                             compute_class_weight(class_weight ='balanced',
                                                  classes = np.unique(y_org),
                                                  y = y_org)))
class_weight_dict

{0: 0.5067785287645394, 1: 37.381159420289855}

# modelling

In [33]:
cate_features_index = np.where(train[features].dtypes != float)[0]
cate_features_index

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17])

In [0]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    class_weights = class_weight_dict,
    iterations = 500, 
    learning_rate = 0.1,
    loss_function = 'CrossEntropy', 
    eval_metric = 'F1',
    use_best_model = True,
    random_seed = 42)

In [41]:
model.fit(train[features],train[y_name],
          cat_features=cate_features_index,
          eval_set=(val_1[features],val_1[y_name]))

0:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 4.98s	remaining: 41m 24s
1:	learn: 0.2702802	test: 0.2802236	best: 0.2802236 (1)	total: 8.24s	remaining: 34m 12s
2:	learn: 0.2176998	test: 0.2253408	best: 0.2802236 (1)	total: 12.4s	remaining: 34m 16s
3:	learn: 0.2265083	test: 0.2347791	best: 0.2802236 (1)	total: 18.1s	remaining: 37m 19s
4:	learn: 0.2131645	test: 0.2272912	best: 0.2802236 (1)	total: 22.4s	remaining: 36m 53s
5:	learn: 0.3050881	test: 0.3095785	best: 0.3095785 (5)	total: 27.4s	remaining: 37m 35s
6:	learn: 0.3201218	test: 0.3233624	best: 0.3233624 (6)	total: 31.7s	remaining: 37m 16s
7:	learn: 0.4193736	test: 0.4268250	best: 0.4268250 (7)	total: 37s	remaining: 37m 53s
8:	learn: 0.4279751	test: 0.4387543	best: 0.4387543 (8)	total: 41.3s	remaining: 37m 34s
9:	learn: 0.4391170	test: 0.4484182	best: 0.4484182 (9)	total: 46.3s	remaining: 37m 49s
10:	learn: 0.4508534	test: 0.4719025	best: 0.4719025 (10)	total: 51.4s	remaining: 38m 5s
11:	learn: 0.4581574	test: 0.4747

# val_2's F1 report

In [42]:
from sklearn.metrics import classification_report 

val_2_pred_01 = pd.Series(model.predict(val_2[features])).apply(round)

print(classification_report(val_2[y_name],val_2_pred_01,target_names=['0','1']))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00    150093
           1       0.79      0.50      0.61      2086

    accuracy                           0.99    152179
   macro avg       0.89      0.75      0.80    152179
weighted avg       0.99      0.99      0.99    152179



# 產生submission

In [43]:
submission = pd.DataFrame({"txkey":test_txkey,
                           "fraud_ind":model.predict(test[features])})

# to [0,1] 
submission["fraud_ind"] = submission["fraud_ind"].apply(round)

# value_counts
print(submission["fraud_ind"].value_counts())
submission.head()

0    418211
1      3454
Name: fraud_ind, dtype: int64


,txkey,fraud_ind
0,592489,0
1,592452,0
2,590212,0
3,590209,0
4,592488,0


# 這裡有個想法如果test上盜刷的比例 跟train上面盜刷的比例 愈相近愈好

In [44]:
train_p = round((train['fraud_ind'].value_counts()[1] / #盜刷樣本數
                len(train['fraud_ind']))*100 ,5) #總數

val_1_p = round((val_1['fraud_ind'].value_counts()[1] / #盜刷樣本數
                len(val_1['fraud_ind']))*100 ,5) #總數

val_2_p = round((val_2['fraud_ind'].value_counts()[1] / #盜刷樣本數
                len(val_2['fraud_ind']))*100 ,5) #總數

test_p = round((submission['fraud_ind'].value_counts()[1] / #盜刷樣本數
                len(submission['fraud_ind']))*100 ,5) #總數
pd.DataFrame({'train_p':train_p,
              'val_1_p':val_1_p,
              'val_2_p':val_2_p,
              'test_p':test_p},
               index=['盜刷比例%'])

,train_p,val_1_p,val_2_p,test_p
盜刷比例%,1.33018,1.36353,1.37075,0.81913


# 保存預測結果

In [0]:
submission.to_csv("./submission_catboost_Fold_0(add_class_weight).csv")